# General Preprocessing: union of different approaches after exploration

Until 13/09/2021 I save the feature extraction for 22KHz and 48 KHz, by using 12 and 40 mel spectrograms and other features. 

In [1]:
# Audio Library
import librosa
# Linear Algebra
import numpy as np
# Handling Paths
import os
# Plots
import matplotlib.pyplot as plt
import librosa.display
# Time
import time
# sklearn
import sklearn

### Paths

In [2]:
root_path = os.getcwd()
print(root_path)

data_path = '../data'

def calling_audios(data_path):
    '''
        Saving audio names
    '''
    names = list()
    for dirname, _, filenames in os.walk(root_path + data_path):
        for filename in filenames:
            names.append(filename)
    return names

audio_names = calling_audios(data_path)

# Audio Numbers
print("Number of total audios: ")
print(len(audio_names))

print("Name Example: ")
print(audio_names[20])

/home/src
Number of total audios: 
1440
Name Example: 
03-01-04-01-01-01-01.wav


In [3]:
def Audio_Energy(x):
    hop_length = 256
    frame_length = 512
    return librosa.feature.rms(x, frame_length=frame_length, hop_length=hop_length, center=True)

def Zero_Crossing(x):
    return librosa.feature.zero_crossing_rate(x)

def Spectral_Centroids(x,sr):
    return librosa.feature.spectral_centroid(x, sr=sr)[0]

def normalize(x, axis=0):
    return sklearn.preprocessing.minmax_scale(x, axis=axis)

### Preprocessing without augmentation

In [4]:
def creating_info_table(path,sample_rate = 48000):
    big_model = []
    small_model = []
    big_model_log = []
    small_model_log = []
    audio_lengths = []
    wave_forms = []
    lst = []
    
    epsilon = 0.01
    
    start_time = time.time()
    count = 0
    for subdir, dirs, files in os.walk(path):
        for file in files:
            count += 1
            try:
                ##Load librosa array, obtain mfcss, store the file and the mcss information in a new array
                X, sample_rate = librosa.load(os.path.join(subdir,file),sr=sample_rate,duration=3, offset=0.5,res_type='kaiser_fast')
                
                
                # make sure waveform vectors are homogenous by defining explicitly
                waveform_homo = np.zeros((int(sample_rate*3,)))
                waveform_homo[:len(X)] = X
                
                # Model 1 (big)
                mfccs = np.mean(librosa.feature.mfcc(y=waveform_homo, sr=sample_rate, n_mfcc=40).T,axis=0) 
                # Model 2 (small)
                mfccs_small = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=12).T,axis=0) 

                # Model 1 (big log) 
                mfccs_log = np.mean(librosa.power_to_db(librosa.feature.mfcc(y=waveform_homo, sr=sample_rate, n_mfcc=40).T),axis=0) 

                # Model 2 (small log) 
                mfccs_small_log = np.mean(librosa.power_to_db(librosa.feature.mfcc(y=waveform_homo, sr=sample_rate, n_mfcc=12).T),axis=0) 

                
                energy = Audio_Energy(waveform_homo)
                zero_cross =  Zero_Crossing(waveform_homo+epsilon)
                spectral_centroids = Spectral_Centroids(waveform_homo+epsilon,sample_rate)
                
                # La lista tiene 3 vectores de caracteristicas por elemento
                arr = energy, zero_cross, spectral_centroids
                lst.append(arr)
                
                
                ## The instruction below converts the labels (from 1 to 8) to a series from 0 to 7
                ## This is because our predictor needs to start from 0 otherwise it will try to predict also 0.
                file = int(file.split("-")[2])
                
                arr = mfccs, file
                arr_small = mfccs_small, file
                arr_log = mfccs_log, file
                arr_small_log = mfccs_small_log, file
                
                big_model.append(arr)
                big_model_log.append(arr_log)
                small_model.append(arr_small)
                small_model_log.append(arr_small_log)
                
                audio_lengths.append(len(waveform_homo))
                wave_forms.append(waveform_homo)
                
          # If the file is not valid, skip it
            except ValueError:
                continue
    print("Total Number of Audio Files")
    print(count)
    print("--- Data loaded. Loading time: %s seconds ---" % (time.time() - start_time))
    
    return big_model, small_model, big_model_log,small_model_log,audio_lengths,wave_forms, lst

### Augmentation by using the waveforms

In [5]:
# RAVDESS native sample rate is 48k
sample_rate = 48000

# Mel Spectrograms are not directly used as a feature in this model
# Mel Spectrograms are used in calculating MFCCs, which are a higher-level representation of pitch transition
# MFCCs work better - left the mel spectrogram function here in case anyone wants to experiment
def feature_melspectrogram(
    waveform, 
    sample_rate,
    fft = 1024,
    winlen = 512,
    window='hamming',
    hop=256,
    mels=128,
    ):
    
    # Produce the mel spectrogram for all STFT frames and get the mean of each column of the resulting matrix to create a feature array
    # Using 8khz as upper frequency bound should be enough for most speech classification tasks
    melspectrogram = librosa.feature.melspectrogram(
        y=waveform, 
        sr=sample_rate, 
        n_fft=fft, 
        win_length=winlen, 
        window=window, 
        hop_length=hop, 
        n_mels=mels, 
        fmax=sample_rate/2)
    
    # convert from power (amplitude**2) to decibels
    # necessary for network to learn - doesn't converge with raw power spectrograms 
    melspectrogram = librosa.power_to_db(melspectrogram, ref=np.max)
    
    return melspectrogram

def feature_mfcc(
    waveform, 
    sample_rate,
    n_mfcc = 12,
    fft = 1024,
    winlen = 512,
    window='hamming',
    #hop=256, # increases # of time steps; was not helpful
    mels=128
    ):

    # Compute the MFCCs for all STFT frames 
    # 40 mel filterbanks (n_mfcc) = 40 coefficients
    mfc_coefficients=librosa.feature.mfcc(
        y=waveform, 
        sr=sample_rate, 
        n_mfcc=n_mfcc,
        n_fft=fft, 
        win_length=winlen, 
        window=window, 
        #hop_length=hop, 
        n_mels=mels, 
        fmax=sample_rate/2
        ) 

    return np.mean(mfc_coefficients.T,axis=0)

def get_features(waveforms, features, samplerate):

    # initialize counter to track progress
    file_count = 0

    # process each waveform individually to get its MFCCs
    for waveform in waveforms:
        mfccs = feature_mfcc(waveform, sample_rate)
        features.append(mfccs)
        file_count += 1
        # print progress 
        print('\r'+f' Processed {file_count}/{len(waveforms)} waveforms',end='')
    
    # return all features from list of waveforms
    return features

def get_waveforms(file):
    
    # load an individual sample audio file
    # read the full 3 seconds of the file, cut off the first 0.5s of silence; native sample rate = 48k
    # don't need to store the sample rate that librosa.load returns
    waveform, _ = librosa.load(file, duration=3, offset=0.5, sr=sample_rate)
    
    # make sure waveform vectors are homogenous by defining explicitly
    waveform_homo = np.zeros((int(sample_rate*3,)))
    waveform_homo[:len(waveform)] = waveform
    
    # return a single file's waveform                                      
    return waveform_homo

In [6]:
def awgn_augmentation(waveform, multiples=2, bits=16, snr_min=15, snr_max=30): 
    
    # get length of waveform (should be 3*48k = 144k)
    wave_len = len(waveform)
    
    # Generate normally distributed (Gaussian) noises
    # one for each waveform and multiple (i.e. wave_len*multiples noises)
    noise = np.random.normal(size=(multiples, wave_len))
    
    # Normalize waveform and noise
    norm_constant = 2.0**(bits-1)
    norm_wave = waveform / norm_constant
    norm_noise = noise / norm_constant
    
    # Compute power of waveform and power of noise
    signal_power = np.sum(norm_wave ** 2) / wave_len
    noise_power = np.sum(norm_noise ** 2, axis=1) / wave_len
    
    # Choose random SNR in decibels in range [15,30]
    snr = np.random.randint(snr_min, snr_max)
    
    # Apply whitening transformation: make the Gaussian noise into Gaussian white noise
    # Compute the covariance matrix used to whiten each noise 
    # actual SNR = signal/noise (power)
    # actual noise power = 10**(-snr/10)
    covariance = np.sqrt((signal_power / noise_power) * 10 ** (- snr / 10))
    # Get covariance matrix with dim: (144000, 2) so we can transform 2 noises: dim (2, 144000)
    covariance = np.ones((wave_len, multiples)) * covariance

    # Since covariance and noise are arrays, * is the haddamard product 
    # Take Haddamard product of covariance and noise to generate white noise
    multiple_augmented_waveforms = waveform + covariance.T * noise
    
    return multiple_augmented_waveforms

def augment_waveforms(waveforms, features, emotions, multiples,n_mfcc = 12):
    # keep track of how many waveforms we've processed so we can add correct emotion label in the same order
    emotion_count = 0
    # keep track of how many augmented samples we've added
    added_count = 0
    # convert emotion array to list for more efficient appending
    emotions = emotions.tolist()
    # Saving augmented waveforms
    lst = []
    epsilon = 0.01

    for waveform in waveforms:

        # Generate 2 augmented multiples of the dataset, i.e. 1440 native + 1440*2 noisy = 4320 samples total
        augmented_waveforms = awgn_augmentation(waveform, multiples=multiples)

        # compute spectrogram for each of 2 augmented waveforms
        for augmented_waveform in augmented_waveforms:

            # Compute MFCCs over augmented waveforms
            # original
            augmented_mfcc = feature_mfcc(augmented_waveform, sample_rate=sample_rate, n_mfcc = n_mfcc)  

            # append the augmented spectrogram to the rest of the native data
            features.append(augmented_mfcc)
            emotions.append(emotions[emotion_count])

            # keep track of new augmented samples
            added_count += 1

            # check progress
            print('\r'+f'Processed {emotion_count + 1}/{len(waveforms)} waveforms for {added_count}/{len(waveforms)*multiples} new augmented samples',end='')
            
            ######################################################
            ### Saving the augmented waveforms
            ######################################################

            waveform_homo = np.zeros((int(sample_rate*3,)))
            waveform_homo[:len(augmented_waveform )] = augmented_waveform 

            energy = Audio_Energy(waveform_homo)
            zero_cross =  Zero_Crossing(waveform_homo+epsilon)
            spectral_centroids = Spectral_Centroids(waveform_homo+epsilon,sample_rate)
            # La lista tiene 3 vectores de caracteristicas por elemento
            arr = energy, zero_cross, spectral_centroids
            lst.append(arr)
            
            
        # keep track of the emotion labels to append in order
        emotion_count += 1
        
        # store augmented waveforms to check their shape
        #augmented_waveforms_temp.append(augmented_waveforms)
    
    return features, emotions,lst

### Loading the original data

In [7]:
big_model, small_model, big_model_log,small_model_log,audio_lengths,wave_forms, lst = creating_info_table(root_path+'/'+data_path)

Total Number of Audio Files
1440
--- Data loaded. Loading time: 191.76410245895386 seconds ---


### Saving

In [8]:
Xb, yb = zip(*big_model)
Xb = np.asarray(Xb)
yb = np.asarray(yb)

Xs, ys = zip(*small_model)
Xs = np.asarray(Xs)
ys = np.asarray(ys)

Xbl, ybl = zip(*big_model_log)
Xbl = np.asarray(Xbl)
ybl = np.asarray(ybl)

Xsl, ysl = zip(*small_model_log)
Xsl = np.asarray(Xsl)
ysl = np.asarray(ysl)

wave_forms = np.asarray(wave_forms)

In [9]:
filename = 'processed_data/mfccs_big_and_small_48.npy'

In [10]:
# open file in write mode and write data
with open(filename, 'wb') as f:
    np.save(f, Xb)
    np.save(f, yb)
    np.save(f, Xs)
    np.save(f, ys)
    np.save(f, Xbl)
    np.save(f, ybl)
    np.save(f, Xsl)
    np.save(f, ysl)
    np.save(f,wave_forms)
    
print(f'Features and labels saved to {filename}')

Features and labels saved to processed_data/mfccs_big_and_small_48.npy


In [11]:
# Check the dimensions of the saved data
print(f'Xb:{Xb.shape}, yb:{yb.shape}')
print(f'Xs:{Xs.shape}, ys:{ys.shape}')
print(f'Xb:{Xbl.shape}, yb:{ybl.shape}')
print(f'Xs:{Xsl.shape}, ys:{ysl.shape}')
print(f'Wave Forms:{wave_forms.shape}')

Xb:(1440, 40), yb:(1440,)
Xs:(1440, 12), ys:(1440,)
Xb:(1440, 40), yb:(1440,)
Xs:(1440, 12), ys:(1440,)
Wave Forms:(1440, 144000)


In [12]:
# open file in read mode and read data 
with open(filename, 'rb') as f:
    Xb = np.load(f)
    yb = np.load(f)
    Xs = np.load(f)
    ys = np.load(f)
    Xbl = np.load(f)
    ybl = np.load(f)
    Xsl = np.load(f)
    ysl = np.load(f)
# Check that we've recovered the right data
print(f'Xb:{Xb.shape}, yb:{yb.shape}')
print(f'Xs:{Xs.shape}, ys:{ys.shape}')
print(f'Xb:{Xbl.shape}, yb:{ybl.shape}')
print(f'Xs:{Xsl.shape}, ys:{ysl.shape}')

Xb:(1440, 40), yb:(1440,)
Xs:(1440, 12), ys:(1440,)
Xb:(1440, 40), yb:(1440,)
Xs:(1440, 12), ys:(1440,)


In [13]:
# saving wave forms

In [14]:
filename = '../processed_data/waveforms_48.npy'
# open file in write mode and write data
with open(filename, 'wb') as f:
    np.save(f, wave_forms)

In [15]:
# saving other features

In [16]:
rms_energy, zero_cross, spectral_centroid = zip(*lst)

In [17]:
filename = '../processed_data/more_features_48.npy'
# open file in write mode and write data
with open(filename, 'wb') as f:
    np.save(f, rms_energy)
    np.save(f, zero_cross)
    np.save(f, spectral_centroid )

print(f'Features and labels saved to {filename}')

Features and labels saved to processed_data/more_features_48.npy


In [18]:
rms_energy_original = np.asarray(rms_energy).copy()
zero_cross_original =  np.asarray(zero_cross).copy()

# Decidí no utilizar el centroide del espectro por ser redundante en relacion al mel spectrogram!
spectral_centroid_original = np.asarray(spectral_centroid).copy()

### Data augmentation and merging the original data!

In [19]:
features_augmented_12 = []

multiples = 3

print('Original waveforms augmented:')
features_augmented_12 , y_augmented_12, lst_augmented_12 = augment_waveforms(wave_forms,features_augmented_12, yb, multiples, n_mfcc = 12)

Original waveforms augmented:
Processed 1440/1440 waveforms for 4320/4320 new augmented samples

In [20]:
print('Original waveforms augmented:')
features_augmented_40 = []
features_augmented_40 , y_augmented_40, lst_augmented_40 = augment_waveforms(wave_forms,features_augmented_40, yb, multiples, n_mfcc = 40)

Original waveforms augmented:
Processed 1440/1440 waveforms for 4320/4320 new augmented samples

### Merging the data into different test groups

For example:
    
    - using 12 bins for the mel-spectrogram with data augmentation
    
    - using 40 bins for the mel-spectrogram with data augmentation
    
    - using 12 bins for the mel-spectrogram with data augmentation + other features
    
    - using 40 bins for the mel-spectrogram with data augmentation + other features

Let us begin with the 12 bin group!: 

In [21]:
print(np.asarray(features_augmented_12).shape)
print(np.asarray(y_augmented_12).shape)  # ya están unidos, el problema es debido a que salvo antes!
print(np.asarray(lst_augmented_12).shape)

(4320, 12)
(5760,)
(4320, 3)


/usr/local/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


Finally, merging: 

In [22]:
F12 = np.vstack((Xs,np.asarray(features_augmented_12)))
F40 = np.vstack((Xb,np.asarray(features_augmented_40)))
Y = np.asarray(y_augmented_12)

In [23]:
print(F12.shape)
print(F40.shape)
print(Y.shape)

(5760, 12)
(5760, 40)
(5760,)


The following matrices are the definitive features. They will be used to find the better architecture.

In [24]:
filename = '../processed_data/MFCCs_AUGMENTED_48.npy'
# open file in write mode and write data
with open(filename, 'wb') as f:
    np.save(f, F12)
    np.save(f, F40)
    np.save(f, Y)

Both, 12 and 40 groups, have the same time and frequency features (I'm not setting the spectrum for this features)

In [25]:
rms_energy, zero_cross, spectral_centroid= zip(*lst_augmented_12)

In [26]:
print(np.asarray(rms_energy).shape)
print(np.asarray(zero_cross).shape)  # ya están unidos, el problema es debido a que salvo antes!
print(np.asarray(spectral_centroid).shape)

(4320, 1, 563)
(4320, 1, 282)
(4320, 282)


In [27]:
print(rms_energy_original.shape)
print(zero_cross_original.shape)  # ya están unidos, el problema es debido a que salvo antes!
print(spectral_centroid_original.shape)

(1440, 1, 563)
(1440, 1, 282)
(1440, 282)


Finally, merging! 

In [28]:
rms_energy = np.reshape(np.asarray(rms_energy),(np.asarray(rms_energy).shape[0],np.asarray(rms_energy).shape[2]))
zero_cross = np.reshape(np.asarray(zero_cross),(np.asarray(zero_cross).shape[0],np.asarray(zero_cross).shape[2]))

In [29]:
print(rms_energy.shape)
print(zero_cross.shape)

(4320, 563)
(4320, 282)


In [30]:
rms_energy_original = np.reshape(np.asarray(rms_energy_original ),(np.asarray(rms_energy_original ).shape[0],np.asarray(rms_energy_original ).shape[2]))
zero_cross_original  = np.reshape(np.asarray(zero_cross_original ),(np.asarray(zero_cross_original ).shape[0],np.asarray(zero_cross_original ).shape[2]))

In [31]:
print(rms_energy_original.shape)
print(zero_cross_original.shape)

(1440, 563)
(1440, 282)


The first 1440 examples are the originals, then the augmented ones. 

In [32]:
E = np.vstack((rms_energy_original,rms_energy))
Z = np.vstack((zero_cross_original,zero_cross))

In [33]:
print(E.shape)
print(Z.shape)

(5760, 563)
(5760, 282)


In [34]:
filename = '../processed_data/ZE_48.npy'
# open file in write mode and write data
with open(filename, 'wb') as f:
    np.save(f, Z)
    np.save(f, E)
    np.save(f, Y)

Merging all the features in one thing!

In [35]:
1440 + multiples*1440

5760

12 features and spectrum!

In [36]:
ALL12_48 = []
for i in range(1440 + multiples*1440):
    ALL12_48.append(np.hstack((E[i,:],Z[i,:],F12[i,:])))

In [37]:
print(np.asarray(ALL12_48).shape)

(5760, 857)


In [38]:
filename = '../processed_data/ALL12_48.npy'
# open file in write mode and write data
with open(filename, 'wb') as f:
    np.save(f, ALL12_48)

In [39]:
ALL40_48 = []
for i in range(1440 + multiples*1440):
    ALL40_48.append(np.hstack((E[i,:],Z[i,:],F40[i,:])))

In [40]:
filename = '../processed_data/ALL40_48.npy'
# open file in write mode and write data
with open(filename, 'wb') as f:
    np.save(f, ALL40_48)

# Preprocesamiento que falta !!!

Terminé de procesar a 48KHz todas las combinaciones de 12,40 + otras caracteristicas! :D

Después tengo que hacer lo mismo para 22050 Hz y también para 16000 Hz, 8000 Hz. 

In [41]:
big_model, small_model, big_model_log,small_model_log,audio_lengths,wave_forms, lst = creating_info_table(root_path+'/'+data_path, 22050)

Total Number of Audio Files
1440
--- Data loaded. Loading time: 165.31732773780823 seconds ---


In [42]:
Xb, yb = zip(*big_model)
Xb = np.asarray(Xb)
yb = np.asarray(yb)

Xs, ys = zip(*small_model)
Xs = np.asarray(Xs)
ys = np.asarray(ys)

Xbl, ybl = zip(*big_model_log)
Xbl = np.asarray(Xbl)
ybl = np.asarray(ybl)

Xsl, ysl = zip(*small_model_log)
Xsl = np.asarray(Xsl)
ysl = np.asarray(ysl)

wave_forms = np.asarray(wave_forms)

In [43]:
filename = '../processed_data/mfccs_big_and_small_22.npy'

In [44]:
# open file in write mode and write data
with open(filename, 'wb') as f:
    np.save(f, Xb)
    np.save(f, yb)
    np.save(f, Xs)
    np.save(f, ys)
    np.save(f, Xbl)
    np.save(f, ybl)
    np.save(f, Xsl)
    np.save(f, ysl)
    np.save(f,wave_forms)
    
print(f'Features and labels saved to {filename}')

Features and labels saved to processed_data/mfccs_big_and_small_22.npy


In [45]:
# Check the dimensions of the saved data
print(f'Xb:{Xb.shape}, yb:{yb.shape}')
print(f'Xs:{Xs.shape}, ys:{ys.shape}')
print(f'Xb:{Xbl.shape}, yb:{ybl.shape}')
print(f'Xs:{Xsl.shape}, ys:{ysl.shape}')
print(f'Wave Forms:{wave_forms.shape}')

Xb:(1440, 40), yb:(1440,)
Xs:(1440, 12), ys:(1440,)
Xb:(1440, 40), yb:(1440,)
Xs:(1440, 12), ys:(1440,)
Wave Forms:(1440, 66150)


In [46]:
filename = '../processed_data/waveforms_22.npy'
# open file in write mode and write data
with open(filename, 'wb') as f:
    np.save(f, wave_forms)

In [47]:
features_augmented_12 = []

multiples = 3

print('Original waveforms augmented:')
features_augmented_12 , y_augmented_12, lst_augmented_12 = augment_waveforms(wave_forms,features_augmented_12, yb, multiples, n_mfcc = 12)

Original waveforms augmented:
Processed 1440/1440 waveforms for 4320/4320 new augmented samples

In [48]:
print('Original waveforms augmented:')
features_augmented_40 = []
features_augmented_40 , y_augmented_40, lst_augmented_40 = augment_waveforms(wave_forms,features_augmented_40, yb, multiples, n_mfcc = 40)

Original waveforms augmented:
Processed 1440/1440 waveforms for 4320/4320 new augmented samples

In [49]:
F12 = np.vstack((Xs,np.asarray(features_augmented_12)))
F40 = np.vstack((Xb,np.asarray(features_augmented_40)))
Y = np.asarray(y_augmented_12)

In [50]:
print(F12.shape)
print(F40.shape)
print(Y.shape)

(5760, 12)
(5760, 40)
(5760,)


In [51]:
filename = '../processed_data/MFCCs_AUGMENTED_22.npy'
# open file in write mode and write data
with open(filename, 'wb') as f:
    np.save(f, F12)
    np.save(f, F40)
    np.save(f, Y)

In [52]:
filename = '../processed_data/more_features_48.npy'
# open file in write mode and write data
with open(filename, 'rb') as f:
    rms_energy = np.load(f)
    zero_cross = np.load(f)
    spectral_centroid = np.load(f)

In [53]:
rms_energy_original = np.asarray(rms_energy).copy()
zero_cross_original =  np.asarray(zero_cross).copy()
spectral_centroid_original = np.asarray(spectral_centroid).copy()

In [54]:
rms_energy_original = np.reshape(np.asarray(rms_energy_original ),(np.asarray(rms_energy_original ).shape[0],np.asarray(rms_energy_original ).shape[2]))
zero_cross_original  = np.reshape(np.asarray(zero_cross_original ),(np.asarray(zero_cross_original ).shape[0],np.asarray(zero_cross_original ).shape[2]))

In [55]:
rms_energy, zero_cross, spectral_centroid= zip(*lst_augmented_12)

In [56]:
rms_energy = np.reshape(np.asarray(rms_energy),(np.asarray(rms_energy).shape[0],np.asarray(rms_energy).shape[2]))
zero_cross = np.reshape(np.asarray(zero_cross),(np.asarray(zero_cross).shape[0],np.asarray(zero_cross).shape[2]))

In [57]:
print(rms_energy_original.shape)
print(zero_cross_original.shape)

(1440, 563)
(1440, 282)


In [58]:
E = np.vstack((rms_energy_original,rms_energy))
Z = np.vstack((zero_cross_original,zero_cross))

In [59]:
print(E.shape)
print(Z.shape)

(5760, 563)
(5760, 282)


In [60]:
filename = '../processed_data/ZE_22.npy'
# open file in write mode and write data
with open(filename, 'wb') as f:
    np.save(f, Z)
    np.save(f, E)
    np.save(f, Y)

In [61]:
ALL12_22 = []
for i in range(1440 + multiples*1440):
    ALL12_22.append(np.hstack((E[i,:],Z[i,:],F12[i,:])))

In [62]:
filename = '../processed_data/ALL12_22.npy'
# open file in write mode and write data
with open(filename, 'wb') as f:
    np.save(f, ALL12_22)

In [63]:
ALL40_22 = []
for i in range(1440 + multiples*1440):
    ALL40_22.append(np.hstack((E[i,:],Z[i,:],F40[i,:])))

In [64]:
filename = '../processed_data/ALL40_22.npy'
# open file in write mode and write data
with open(filename, 'wb') as f:
    np.save(f, ALL40_22)